In [1]:
from CBD.Core import CBD
from CBD.lib.std import *

class Train(CBD):
    def __init__(self, name, dt):
        super().__init__(name, ["a", "v0", "x0"], ["v", "x"])
        
        self.addBlock(ConstantBlock("dt", dt))
        self.addBlock(IntegratorBlock("vI"))
        self.addBlock(IntegratorBlock("xI"))
        self.addConnection("a", "vI")
        self.addConnection("vI", "xI")
        self.addConnection("xI", "x")
        self.addConnection("vI", "v")
        self.addConnection("dt", "xI", input_port_name="delta_t")
        self.addConnection("dt", "vI", input_port_name="delta_t")
        self.addConnection("v0", "vI", input_port_name="IC")
        self.addConnection("x0", "xI", input_port_name="IC")

In [3]:
from pypdevs.DEVS import AtomicDEVS, CoupledDEVS
from pypdevs.infinity import INFINITY

class Compute(AtomicDEVS):
    def __init__(self, name):
        super().__init__(name)
        
        self.state = {
            "speeding": True,
            "braking": False,
            "acceleration": 10,
            "output_requested": True
        }
        
        self.crossing = self.addInPort("crossing")
        self.acc = self.addOutPort("acc")
        
    def extTransition(self, inputs):
        crossing = inputs[self.crossing]
        if "v" in crossing and not self.state["braking"]:
            self.speed["speeding"] = not self.speed["speeding"]
            if self.speed["speeding"]:
                a = 10
            else:
                a = -5
        if "brake" in crossing:
            a = -5
            self.state["braking"] = True
        if "station" in crossing:
            a = 0
        self.state["acceleration"] = a
        self.state["output_requested"] = True
        return self.state
    
    def outputFnc(self):
        return {
            self.acc: self.state["acceleration"]
        }
    
    def intTransition(self):
        self.state["output_requested"] = False
        return self.state
    
    def timeAdvance(self):
        if self.state["output_requested"]:
            return 0.0
        return INFINITY
        
        

In [ ]:
from CBD.converters.cbddevs import CBDRunner, CrossingDetection

class TrainSystem(CoupledDEVS):
    def __init__(self, name):
        super().__init__(name)
        
        self.driver = self.addSubModel(Compute("Driver"))
        self.train = self.addSubModel(CBDRunner("train", Train("train", 0.1), {"a": 10, "v0": 0, "x0": 0}))
        
        